In [1]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
import cv2
import os

In [2]:
# Load pretrained models
path_to_time_model = r"C:\Users\josep\Python Projects\NN for clock\Trained Models\time_model05_more_data"
time_model = tf.saved_model.load(path_to_time_model)
path_to_quarter_model = r"C:\Users\josep\Python Projects\NN for clock\Trained Models\quarter_model02"
quarter_model = tf.saved_model.load(path_to_quarter_model)

# Lists of target values and quarters
target_values = np.array([38, 38, 38, 43, 124, 220, 220, 230, 318, 321, 351, 351, 358, 406, 452, 523, 557, 602, 602, 602, 602, 610, 650, 735, 818, 849, 937, 1010, 1048, 1131, 1152, 1237, 1301, 1315, 1350, 1350, 1500, 1500, 0, 1, 1, 9, 13, 18, 23, 23, 28, 32, 55, 55, 100, 110, 115, 120, 135, 135, 150, 214, 255, 331, 331, 331, 331, 337, 400, 437, 515, 540, 606, 628, 701, 731, 813, 839, 839, 921, 929, 931, 931, 931, 942, 1010, 1038, 1110, 1153, 1216, 1235, 1312, 1339, 1413, 1456, 1500, 20, 54, 121, 154, 221, 255, 303, 303, 303, 303, 309, 350, 425, 431, 506, 536, 559, 642, 642, 642, 642, 647, 720, 746, 817, 822, 906, 951, 1016, 1016, 1023, 1103, 1140, 1200, 1234, 1242, 1319, 1319, 1327, 1327, 1333, 1344, 1415, 1500, 1500, 10, 10, 57, 57, 101, 101, 107, 108, 108, 108, 111, 129, 134, 151, 213, 222, 238, 302, 325, 328, 355, 405, 405, 413, 418, 426, 426, 430, 436, 502, 524, 537, 603, 622, 701, 707, 743, 817, 856, 901, 921, 956, 1006, 1046, 1109, 1139, 1149, 1152, 1213, 1247, 1324, 1324, 1324, 1329, 1411, 1418, 1422, 1500])
# target_values = target_values[::-1]
target_quarters = np.array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4])

# Video path
video_path = r"D:\Videos\23' Arizona Football Games\Football 7 Washington at Arizona 93023 Pac-12.mp4"

# Open video
cap = cv2.VideoCapture(video_path)

# Get video properties
frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
video_fps = cap.get(cv2.CAP_PROP_FPS)
video_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))

In [ ]:
import cv2
import os
import numpy as np
import tensorflow as tf
from concurrent.futures import ThreadPoolExecutor, as_completed
import gc
import time
import subprocess

# Initialize variables for frame processing
found_match = False
num_matches_found = 0

# Directory where the videos will be saved
output_directory = r"C:\Users\josep\Python Projects\Output Videos\Washington Plays (Sofascore)"
if not os.path.exists(output_directory):
    os.makedirs(output_directory)

# Initialize video writer object for saving the matched segment
fourcc = cv2.VideoWriter_fourcc(*'mp4v')

# Function to process a single frame
def predict_frame(cropped_frame):
    cropped_frame = np.array(cropped_frame)
    prediction_frame = cropped_frame.astype('float32') / 255.0
    prediction = time_model(np.expand_dims(prediction_frame, axis=0))
    prediction = tf.argmax(prediction, axis=1).numpy()[0]
    return prediction

def predict_quarter_frame(cropped_quarter_frame):
    cropped_quarter_frame = np.array(cropped_quarter_frame)
    prediction_quarter_frame = cropped_quarter_frame.astype('float32') / 255.0
    prediction_quarter = quarter_model(np.expand_dims(prediction_quarter_frame, axis=0))
    prediction_quarter = tf.argmax(prediction_quarter, axis=1).numpy()[0]
    return prediction_quarter

def process_frame(frame_data):
    frame, target_values, target_quarters = frame_data
    frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    
    # Crop frame using NumPy slicing
    x1, y1 = 1320, 915
    x2, y2 = 1460, 965
    cropped_frame = frame_rgb[y1:y2, x1:x2]
    
    prediction = predict_frame(cropped_frame)
    
    for target_value, target_quarter in zip(target_values, target_quarters):
        if prediction == target_value:
            # Crop quarter frame for quarter model
            x1_quarter, y1_quarter = 1257, 915
            x2_quarter, y2_quarter = 1344, 965
            cropped_quarter_frame = frame_rgb[y1_quarter:y2_quarter, x1_quarter:x2_quarter]
            
            prediction_quarter = predict_quarter_frame(cropped_quarter_frame)
            
            if prediction_quarter == target_quarter:
                print(f"Match found: Prediction {prediction}, Quarter {prediction_quarter}")
                return prediction, prediction_quarter, frame_rgb, cropped_frame, target_value, target_quarter

    return None

# Function to save video clips
def save_clip(frames, output_video_path, video_fps, frame_width, frame_height):
    print(f"Saving clip to {output_video_path}")
    output_video = cv2.VideoWriter(output_video_path, fourcc, video_fps, (frame_width, frame_height))
    for frame in frames:
        output_video.write(frame)
    output_video.release()
    print(f"Clip saved to {output_video_path}")

# Function to download audio file with ffmpeg
def download_audio_segment(video_path, output_audio_path, start_time, duration):
    print(f"Downloading audio segment from {start_time} seconds to {start_time + duration} seconds")
    command = [
        'ffmpeg',
        '-i', video_path,
        '-ss', str(start_time),  # Start time
        '-t', str(duration),  # Duration
        '-vn',  # No video
        '-acodec', 'copy',  # Copy audio codec
        output_audio_path
    ]
    subprocess.run(command, check=True)
    print(f"Audio segment downloaded to {output_audio_path}")

# Function to process video frames in parallel
def process_video_parallel(video_path, target_values, target_quarters, video_fps, frame_width, frame_height, batch_size=500):
    matches = {}
    cap = cv2.VideoCapture(video_path)
    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    print(f"Total frames in video: {total_frames}")
    cap.release()

    def process_batch(video_path, start_frame, end_frame):
        start_time = time.time()
        local_matches = {}
        cap = cv2.VideoCapture(video_path)
        cap.set(cv2.CAP_PROP_POS_FRAMES, start_frame)
        frame_idx = start_frame
        while frame_idx < end_frame:
            ret, frame = cap.read()
            if not ret:
                break
            frame_data = (frame, target_values, target_quarters)
            match = process_frame(frame_data)
            if match:
                prediction, prediction_quarter, frame_rgb, cropped_frame, target_value, target_quarter = match
                key = (prediction, prediction_quarter)
                local_matches[key] = (frame_rgb, cropped_frame, target_value, target_quarter, frame_idx)
                frame_idx += 1
            #else:
                #frame_idx += 30  # Skip the next 29 frames
                #cap.set(cv2.CAP_PROP_POS_FRAMES, frame_idx)
        
        cap.release()
        end_time = time.time()
        print(f"Processed batch from {start_frame} to {end_frame} in {end_time - start_time:.2f} seconds")
        return local_matches

    with ThreadPoolExecutor(max_workers=16) as executor:
        futures = []
        batch_start_time = time.time()
        for start_frame in range(0, total_frames, batch_size):
            end_frame = min(start_frame + batch_size, total_frames)
            futures.append(executor.submit(process_batch, video_path, start_frame, end_frame))

        for future in as_completed(futures):
            local_matches = future.result()
            matches.update(local_matches)

    total_time = time.time() - batch_start_time
    print(f"Total time to process video: {total_time:.2f} seconds")
    print(f"Total matches found: {len(matches)}")
    return matches

# Main processing loop
def main(video_path, target_values, target_quarters, frame_width, frame_height):
    global num_matches_found, found_match
    
    print("Starting video processing")
    
    # Check if video file exists
    if not os.path.exists(video_path):
        print(f"Error: Video file '{video_path}' does not exist.")
        return

    cap = cv2.VideoCapture(video_path)
    
    # Check if the video was opened successfully
    if not cap.isOpened():
        print(f"Error: Could not open video file '{video_path}'.")
        return
    
    # Get the FPS of the video
    video_fps = cap.get(cv2.CAP_PROP_FPS)
    if video_fps == 0:
        print("Error: Could not get FPS of the video.")
        return
    print(f"Frames per second: {video_fps}")
    
    matches = process_video_parallel(video_path, target_values, target_quarters, video_fps, frame_width, frame_height)
    
    cap.release()  # Release capture resource after frames are read
    
    with ThreadPoolExecutor(max_workers=16) as executor:
        future_save_clips = []
        for (prediction, prediction_quarter), (frame_rgb, cropped_frame, target_value, target_quarter, frame_count) in matches.items():
            output_video_name = f"{prediction}_{prediction_quarter}.mp4"
            output_video_path = os.path.join(output_directory, output_video_name)
            output_audio_path = os.path.join(output_directory, f"{prediction}_{prediction_quarter}.aac")
            
            start_frame_index = max(frame_count - int(1 * video_fps), 0)
            end_frame_index = start_frame_index + int(8 * video_fps)
            frames = []

            cap = cv2.VideoCapture(video_path)
            cap.set(cv2.CAP_PROP_POS_FRAMES, start_frame_index)
            print(f"Extracting frames from {start_frame_index} to {end_frame_index}")
            while cap.get(cv2.CAP_PROP_POS_FRAMES) < end_frame_index:
                ret, frame = cap.read()
                if not ret:
                    break
                frames.append(frame)
            cap.release()
            
            future_save_clips.append(executor.submit(save_clip, frames, output_video_path, video_fps, frame_width, frame_height))
            
            # Calculate start time and duration for audio extraction
            start_time = start_frame_index / video_fps
            duration = (end_frame_index - start_frame_index) / video_fps
            
            # Download audio segment for the match
            download_audio_segment(video_path, output_audio_path, start_time, duration)
            
            num_matches_found += 1
            gc.collect()  # Force garbage collection to free up memory
            
            if num_matches_found == len(target_values):
                found_match = True
                break
        
        for future in as_completed(future_save_clips):
            future.result()  # Wait for all save operations to complete

    if found_match:
        print("All target matches found")
        plt.imshow(cropped_frame)
        plt.title("Cropped Frame")
        plt.axis('off')
        plt.show()

# Run the main function
main(video_path, target_values, target_quarters, frame_width, frame_height)

print("Done")

Starting video processing
Frames per second: 29.9703554996489
Total frames in video: 82869
Match found: Prediction 1500, Quarter 1
Match found: Prediction 1500, Quarter 1
Match found: Prediction 1500, Quarter 1
Match found: Prediction 1500, Quarter 1
Match found: Prediction 1301, Quarter 1
Match found: Prediction 1500, Quarter 1
Match found: Prediction 1301, Quarter 1
Match found: Prediction 1500, Quarter 1
Match found: Prediction 1301, Quarter 1
Match found: Prediction 1500, Quarter 1
Match found: Prediction 1301, Quarter 1
Match found: Prediction 1500, Quarter 1
Match found: Prediction 1301, Quarter 1
Match found: Prediction 1500, Quarter 1
Match found: Prediction 1301, Quarter 1
Match found: Prediction 1500, Quarter 1
Match found: Prediction 1301, Quarter 1
Match found: Prediction 1500, Quarter 1
Match found: Prediction 1301, Quarter 1
Match found: Prediction 1500, Quarter 1
Match found: Prediction 1301, Quarter 1
Match found: Prediction 1500, Quarter 1
Match found: Prediction 1301,

## Works on Wazzu

In [ ]:
import cv2
import os
import numpy as np
import tensorflow as tf
from concurrent.futures import ThreadPoolExecutor, as_completed
import gc
import time

# Initialize variables for frame processing
found_match = False
num_matches_found = 0

# Directory where the videos will be saved
output_directory = r"C:\Users\josep\Python Projects\Output Videos\Utah Key Plays (Sofascore)"
if not os.path.exists(output_directory):
    os.makedirs(output_directory)

# Initialize video writer object for saving the matched segment
fourcc = cv2.VideoWriter_fourcc(*'mp4v')

# Function to process a single frame
def predict_frame(cropped_frame):
    cropped_frame = np.array(cropped_frame)
    prediction_frame = cropped_frame.astype('float32') / 255.0
    prediction = time_model(np.expand_dims(prediction_frame, axis=0))
    prediction = tf.argmax(prediction, axis=1).numpy()[0]
    return prediction

def predict_quarter_frame(cropped_quarter_frame):
    cropped_quarter_frame = np.array(cropped_quarter_frame)
    prediction_quarter_frame = cropped_quarter_frame.astype('float32') / 255.0
    prediction_quarter = quarter_model(np.expand_dims(prediction_quarter_frame, axis=0))
    prediction_quarter = tf.argmax(prediction_quarter, axis=1).numpy()[0]
    return prediction_quarter

def process_frame(frame_data):
    frame, target_values, target_quarters = frame_data
    frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    
    # Crop frame using NumPy slicing
    x1, y1 = 1320, 915
    x2, y2 = 1460, 965
    cropped_frame = frame_rgb[y1:y2, x1:x2]
    
    prediction = predict_frame(cropped_frame)
    
    for target_value, target_quarter in zip(target_values, target_quarters):
        if prediction == target_value:
            # Crop quarter frame for quarter model
            x1_quarter, y1_quarter = 1257, 915
            x2_quarter, y2_quarter = 1344, 965
            cropped_quarter_frame = frame_rgb[y1_quarter:y2_quarter, x1_quarter:x2_quarter]
            
            prediction_quarter = predict_quarter_frame(cropped_quarter_frame)
            
            if prediction_quarter == target_quarter:
                print(f"Match found: Prediction {prediction}, Quarter {prediction_quarter}")
                return prediction, prediction_quarter, frame_rgb, cropped_frame, target_value, target_quarter

    return None

# Function to save video clips
def save_clip(frames, output_video_path, video_fps, frame_width, frame_height):
    print(f"Saving clip to {output_video_path}")
    output_video = cv2.VideoWriter(output_video_path, fourcc, video_fps, (frame_width, frame_height))
    for frame in frames:
        output_video.write(frame)
    output_video.release()
    print(f"Clip saved to {output_video_path}")

# Function to process video frames in parallel
def process_video_parallel(video_path, target_values, target_quarters, video_fps, frame_width, frame_height, batch_size=500):
    matches = {}
    cap = cv2.VideoCapture(video_path)
    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    print(f"Total frames in video: {total_frames}")
    cap.release()

    def process_batch(video_path, start_frame, end_frame):
        start_time = time.time()
        local_matches = {}
        cap = cv2.VideoCapture(video_path)
        cap.set(cv2.CAP_PROP_POS_FRAMES, start_frame)
        frame_idx = start_frame
        while frame_idx < end_frame:
            ret, frame = cap.read()
            if not ret:
                break
            frame_data = (frame, target_values, target_quarters)
            match = process_frame(frame_data)
            if match:
                prediction, prediction_quarter, frame_rgb, cropped_frame, target_value, target_quarter = match
                key = (prediction, prediction_quarter)
                local_matches[key] = (frame_rgb, cropped_frame, target_value, target_quarter, frame_idx)
                frame_idx += 1
            else:
                frame_idx += 30  # Skip the next 29 frames
                cap.set(cv2.CAP_PROP_POS_FRAMES, frame_idx)
        
        cap.release()
        end_time = time.time()
        print(f"Processed batch from {start_frame} to {end_frame} in {end_time - start_time:.2f} seconds")
        return local_matches

    with ThreadPoolExecutor(max_workers=16) as executor:
        futures = []
        batch_start_time = time.time()
        for start_frame in range(0, total_frames, batch_size):
            end_frame = min(start_frame + batch_size, total_frames)
            futures.append(executor.submit(process_batch, video_path, start_frame, end_frame))

        for future in as_completed(futures):
            local_matches = future.result()
            matches.update(local_matches)

    total_time = time.time() - batch_start_time
    print(f"Total time to process video: {total_time:.2f} seconds")
    print(f"Total matches found: {len(matches)}")
    return matches

# Main processing loop
def main(video_path, target_values, target_quarters, frame_width, frame_height):
    global num_matches_found, found_match
    
    print("Starting video processing")
    
    # Check if video file exists
    if not os.path.exists(video_path):
        print(f"Error: Video file '{video_path}' does not exist.")
        return

    cap = cv2.VideoCapture(video_path)
    
    # Check if the video was opened successfully
    if not cap.isOpened():
        print(f"Error: Could not open video file '{video_path}'.")
        return
    
    # Get the FPS of the video
    video_fps = cap.get(cv2.CAP_PROP_FPS)
    if video_fps == 0:
        print("Error: Could not get FPS of the video.")
        return
    print(f"Frames per second: {video_fps}")
    
    matches = process_video_parallel(video_path, target_values, target_quarters, video_fps, frame_width, frame_height)
    
    cap.release()  # Release capture resource after frames are read
    
    with ThreadPoolExecutor(max_workers=16) as executor:
        future_save_clips = []
        for (prediction, prediction_quarter), (frame_rgb, cropped_frame, target_value, target_quarter, frame_count) in matches.items():
            output_video_name = f"{prediction}_{prediction_quarter}.mp4"
            output_video_path = os.path.join(output_directory, output_video_name)
            
            start_frame_index = max(frame_count - int(5 * video_fps), 0)
            end_frame_index = start_frame_index + int(20 * video_fps)
            frames = []

            cap = cv2.VideoCapture(video_path)
            cap.set(cv2.CAP_PROP_POS_FRAMES, start_frame_index)
            print(f"Extracting frames from {start_frame_index} to {end_frame_index}")
            while cap.get(cv2.CAP_PROP_POS_FRAMES) < end_frame_index:
                ret, frame = cap.read()
                if not ret:
                    break
                frames.append(frame)
            cap.release()
            
            future_save_clips.append(executor.submit(save_clip, frames, output_video_path, video_fps, frame_width, frame_height))
            
            num_matches_found += 1
            gc.collect()  # Force garbage collection to free up memory
            
            if num_matches_found == len(target_values):
                found_match = True
                break
        
        for future in as_completed(future_save_clips):
            future.result()  # Wait for all save operations to complete

    if found_match:
        print("All target matches found")
        plt.imshow(cropped_frame)
        plt.title("Cropped Frame")
        plt.axis('off')
        plt.show()

# Run the main function
main(video_path, target_values, target_quarters, frame_width, frame_height)

Starting video processing
Frames per second: 30.0
Total frames in video: 473206
Match found: Prediction 1033, Quarter 1
Match found: Prediction 1033, Quarter 1
Match found: Prediction 1033, Quarter 1
Match found: Prediction 1033, Quarter 1
Match found: Prediction 1033, Quarter 1
Match found: Prediction 1033, Quarter 1
Match found: Prediction 1033, Quarter 1
Match found: Prediction 1033, Quarter 1
Match found: Prediction 1033, Quarter 1
Match found: Prediction 1033, Quarter 1
Match found: Prediction 1033, Quarter 1
Match found: Prediction 1033, Quarter 1
Match found: Prediction 1033, Quarter 1
Processed batch from 500 to 1000 in 3.54 seconds
Match found: Prediction 1033, Quarter 1
Match found: Prediction 1033, Quarter 1
Match found: Prediction 1033, Quarter 1
Match found: Prediction 1033, Quarter 1
Match found: Prediction 1033, Quarter 1
Match found: Prediction 1033, Quarter 1
Match found: Prediction 1033, Quarter 1
Match found: Prediction 1033, Quarter 1
Match found: Prediction 1033, Q